In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

spark = SparkSession.builder.appName("Ingesta Silver") \
    .master("spark://localhost:7077")\
    .config("spark.jars.packages", "io.delta:delta-core_2.12:0.8.0") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.sql.sources.partitionOverwriteMode", "dynamic")\
    .config("hive.exe.dynamic.partition.mode", "nonstrict") \
    .getOrCreate()

from delta.tables import *

24/10/07 20:59:22 WARN Utils: Your hostname, besgam-Aspire-E1-471 resolves to a loopback address: 127.0.1.1; using 192.168.0.21 instead (on interface wlp9s0)
24/10/07 20:59:22 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Ivy Default Cache set to: /home/besgam/.ivy2/cache
The jars for the packages stored in: /home/besgam/.ivy2/jars
:: loading settings :: url = jar:file:/home/besgam/anaconda3/lib/python3.8/site-packages/pyspark/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-ea524069-8fad-4624-8bd9-e2611a540faf;1.0
	confs: [default]
	found io.delta#delta-core_2.12;0.8.0 in central
	found org.antlr#antlr4;4.7 in central
	found org.antlr#antlr4-runtime;4.7 in central
	found org.antlr#antlr-runtime;3.5.2 in central
	found org.antlr#ST4;4.0.8 in central
	found org.abego.treelayout#org.abego.treelayout.core;1.0.3 in central
	found org.glass

In [2]:
anoMes = "'2024-06'"

In [6]:
anoMes

"'2024-05'"

In [3]:
def qualifyTypeColumn(df, columns, expressions):
    for column, expression in zip(columns, expressions):
        df = df.withColumn(column, expression)
    return df

In [4]:
columns = ['dataAbertura', 'dataResposta', 'dataAnalise', 'dataRecusa', 'prazoResposta', 'procurouEmpresa', 'respondida']

In [5]:
expressions = [ from_unixtime(unix_timestamp(col('dataAbertura'), 'dd/MM/yyyy')).cast(DateType())
               , from_unixtime(unix_timestamp(col('dataResposta'), 'dd/MM/yyyy')).cast(DateType()) 
               , from_unixtime(unix_timestamp(col('dataAnalise'), 'dd/MM/yyyy')).cast(DateType()) 
               , from_unixtime(unix_timestamp(col('dataRecusa'), 'dd/MM/yyyy')).cast(DateType())               
               , from_unixtime(unix_timestamp(col('prazoResposta'), 'dd/MM/yyyy')).cast(DateType())
               , when(col('procurouEmpresa') == 'S', lit(1)).otherwise(lit(0)).cast(BooleanType())
               , when(col('respondida') == 'S', lit(1)).otherwise(lit(0)).cast(BooleanType())
              ]

In [2]:
consumidor = spark.read.format('delta').load("hdfs://namenode:9000/data/consumidor/bronze/")#.where(f"datRefCarga!={anoMes}")

In [3]:
maxPartition = spark.read.format('delta') \
                       .load("hdfs://namenode:9000/data/consumidor/bronze/")\
                       .select("datRefCarga").orderBy(col("datProc").desc()).first()[0]

24/09/24 20:52:46 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [4]:
maxPartition

'2024-06'

In [7]:
df = spark.read.format('delta') \
                       .load("hdfs://namenode:9000/data/consumidor/bronze/") \
                       .where(f"datRefCarga='{maxPartition}'")

In [20]:
consumidor = qualifyTypeColumn(consumidor, columns, expressions)

In [21]:
consumidor = consumidor.filter((col('nomefantasia').like('Banco%')) & (col('area')=='Serviços Financeiros') )

In [22]:
consumidor.write.partitionBy('datRefCarga').format("delta").mode("append").save("hdfs://namenode:9000/data/consumidor/silver")

24/08/15 21:34:03 ERROR StandaloneSchedulerBackend: Application has been killed. Reason: Master removed our application: KILLED
24/08/15 21:34:03 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exiting due to error from cluster scheduler: Master removed our application: KILLED
	at org.apache.spark.scheduler.TaskSchedulerImpl.error(TaskSchedulerImpl.scala:716)
	at org.apache.spark.scheduler.cluster.StandaloneSchedulerBackend.dead(StandaloneSchedulerBackend.scala:152)
	at org.apache.spark.deploy.client.StandaloneAppClient$ClientEndpoint.markDead(StandaloneAppClient.scala:258)
	at org.apache.spark.deploy.client.StandaloneAppClient$ClientEndpoint$$anonfun$receive$1.applyOrElse(StandaloneAppClient.scala:168)
	at org.apache.spark.rpc.netty.Inbox.$anonfun$process$1(Inbox.scala:115)
	at org.apache.spark.rpc.netty.Inbox.safelyCall(Inbox.scala:203)
	at org.apache.spark.rpc.netty.Inbox.process(Inbox.scala:100)
	at org.apache.spark.rpc.netty.MessageLoop.org$apache$spark$rpc$netty$Mess

In [3]:
df = spark.read.parquet('hdfs://namenode:9000/data/consumidor/silver/datRefCarga=2023-05')

In [4]:
df.printSchema()

root
 |-- gestor: string (nullable = true)
 |-- canalOrigem: string (nullable = true)
 |-- regiao: string (nullable = true)
 |-- uf: string (nullable = true)
 |-- cidade: string (nullable = true)
 |-- sexo: string (nullable = true)
 |-- faixaEtaria: string (nullable = true)
 |-- anoAbertura: integer (nullable = true)
 |-- mesAbertura: integer (nullable = true)
 |-- dataAbertura: date (nullable = true)
 |-- dataResposta: date (nullable = true)
 |-- dataAnalise: date (nullable = true)
 |-- dataRecusa: date (nullable = true)
 |-- dataFinalizacao: date (nullable = true)
 |-- prazoResposta: date (nullable = true)
 |-- prazoAnaliseGestor: integer (nullable = true)
 |-- tempoResposta: string (nullable = true)
 |-- nomeFantasia: string (nullable = true)
 |-- segmentoMercado: string (nullable = true)
 |-- area: string (nullable = true)
 |-- assunto: string (nullable = true)
 |-- grupoProblema: string (nullable = true)
 |-- problema: string (nullable = true)
 |-- comoContratou: string (nullable 

In [5]:
df.show(10)

24/10/07 21:00:52 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+--------------------+--------------+------+---+--------------------+----+------------------+-----------+-----------+------------+------------+-----------+----------+---------------+-------------+------------------+-------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+---------------+----------+--------------------+-------------------+--------------+-------------+---------------------------+
|              gestor|   canalOrigem|regiao| uf|              cidade|sexo|       faixaEtaria|anoAbertura|mesAbertura|dataAbertura|dataResposta|dataAnalise|dataRecusa|dataFinalizacao|prazoResposta|prazoAnaliseGestor|tempoResposta|        nomeFantasia|     segmentoMercado|                area|             assunto|       grupoProblema|            problema|       comoContratou|procurouEmpresa|respondida|            situacao|avaliacaoReclamacao|notaConsumidor|analiseRecusa|tempoRespostanotaConsumid

24/10/07 21:10:35 WARN HeartbeatReceiver: Removing executor 0 with no recent heartbeats: 121536 ms exceeds timeout 120000 ms
24/10/07 21:10:35 ERROR TaskSchedulerImpl: Lost executor 0 on 172.18.0.10: Executor heartbeat timed out after 121536 ms
24/10/07 21:11:01 WARN TransportChannelHandler: Exception in connection from /172.18.0.10:58360
java.io.IOException: Connection reset by peer
	at java.base/sun.nio.ch.FileDispatcherImpl.read0(Native Method)
	at java.base/sun.nio.ch.SocketDispatcher.read(SocketDispatcher.java:39)
	at java.base/sun.nio.ch.IOUtil.readIntoNativeBuffer(IOUtil.java:276)
	at java.base/sun.nio.ch.IOUtil.read(IOUtil.java:233)
	at java.base/sun.nio.ch.IOUtil.read(IOUtil.java:223)
	at java.base/sun.nio.ch.SocketChannelImpl.read(SocketChannelImpl.java:356)
	at io.netty.buffer.PooledByteBuf.setBytes(PooledByteBuf.java:253)
	at io.netty.buffer.AbstractByteBuf.writeBytes(AbstractByteBuf.java:1133)
	at io.netty.channel.socket.nio.NioSocketChannel.doReadBytes(NioSocketChannel.ja